<a href="https://colab.research.google.com/github/Leos1999/Hackathons/blob/master/Traffic_volume(Hackathon).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
data = pd.read_csv('drive/My Drive/Datasets/Predict_traffic(Hackathon)/DataSets/Train.csv')

In [0]:
data.head()


,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume
0,2012-10-02 09:00:00,None,121,89,2,329,1,1,288.28,0.0,0.0,40,Clouds,scattered clouds,5545
1,2012-10-02 10:00:00,None,178,67,3,330,1,1,289.36,0.0,0.0,75,Clouds,broken clouds,4516
2,2012-10-02 11:00:00,None,113,66,3,329,2,2,289.58,0.0,0.0,90,Clouds,overcast clouds,4767
3,2012-10-02 12:00:00,None,20,66,3,329,5,5,290.13,0.0,0.0,90,Clouds,overcast clouds,5026
4,2012-10-02 13:00:00,None,281,65,3,329,7,7,291.14,0.0,0.0,75,Clouds,broken clouds,4918


In [0]:
data.is_holiday.unique()

array(['None', 'Columbus Day', 'Veterans Day', 'Thanksgiving Day',
       'Christmas Day', 'New Years Day', 'Washingtons Birthday',
       'Memorial Day', 'Independence Day', 'State Fair', 'Labor Day',
       'Martin Luther King Jr Day'], dtype=object)

In [0]:
data.is_holiday.replace(['None', 'Columbus Day', 'Veterans Day', 'Thanksgiving Day',
       'Christmas Day', 'New Years Day', 'Washingtons Birthday',
       'Memorial Day', 'Independence Day', 'State Fair', 'Labor Day',
       'Martin Luther King Jr Day'],[0,1,1,1,1,1,1,1,1,1,1,1],inplace=True)

In [0]:
# encoder = preprocessing.LabelEncoder()
# encoder.fit(data['is_holiday'])
# data['is_holiday']= encoder.transform(data['is_holiday'])
data.tail()


,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume
33745,2017-05-17 21:00:00,0,74,85,1,328,7,7,287.88,0.0,0.0,90,Mist,mist,2348
33746,2017-05-17 22:00:00,0,81,70,1,24,2,2,286.95,0.0,0.0,90,Rain,heavy intensity rain,2194
33747,2017-05-17 22:00:00,0,138,70,1,24,1,1,286.95,0.0,0.0,90,Mist,mist,2194
33748,2017-05-17 23:00:00,0,229,66,1,42,6,6,285.75,0.0,0.0,90,Rain,heavy intensity rain,1328
33749,2017-05-17 23:00:00,0,140,63,1,27,8,8,285.75,0.0,0.0,90,Mist,mist,1328


In [0]:
data.isnull().sum()
print(len(data))

33750


In [0]:
data['weather_description'].replace(['light intensity shower rain','light rain and snow','SQUALLS','light shower snow','thunderstorm with light drizzle','sleet','freezing rain','thunderstorm with drizzle','shower drizzle','shower snow'],['Others','Others','Others','Others','Others','Others','Others','Others','Others','Others'],inplace=True)

In [0]:
data.weather_description.value_counts()

sky is clear                           7524
mist                                   4047
overcast clouds                        3972
broken clouds                          3379
light rain                             2442
scattered clouds                       2294
Sky is Clear                           1709
few clouds                             1568
light snow                             1280
moderate rain                          1105
haze                                    902
light intensity drizzle                 703
heavy snow                              587
fog                                     567
drizzle                                 460
proximity thunderstorm                  345
heavy intensity rain                    311
snow                                    190
thunderstorm                             66
proximity shower rain                    54
thunderstorm with heavy rain             42
heavy intensity drizzle                  40
thunderstorm with light rain    

In [0]:
data.weather_type.value_counts()

Clouds          11213
Clear            9233
Mist             4047
Rain             3940
Snow             2070
Drizzle          1204
Haze              902
Fog               567
Thunderstorm      553
Smoke              17
Squall              4
Name: weather_type, dtype: int64

In [0]:
data.weather_type.replace(['Smoke','Squall'],['Others','Others'],inplace=True)

In [0]:
df = pd.get_dummies(data, prefix_sep="__",
                              columns=['weather_type','weather_description'])


In [0]:
df.head()

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,traffic_volume,weather_type__Clear,weather_type__Clouds,weather_type__Drizzle,weather_type__Fog,weather_type__Haze,weather_type__Mist,weather_type__Others,weather_type__Rain,weather_type__Snow,weather_type__Thunderstorm,weather_description__Others,weather_description__Sky is Clear,weather_description__broken clouds,weather_description__drizzle,weather_description__few clouds,weather_description__fog,weather_description__haze,weather_description__heavy intensity drizzle,weather_description__heavy intensity rain,weather_description__heavy snow,weather_description__light intensity drizzle,weather_description__light rain,weather_description__light snow,weather_description__mist,weather_description__moderate rain,weather_description__overcast clouds,weather_description__proximity shower rain,weather_description__proximity thunderstorm,weather_description__proximity thunderstorm with drizzle,weather_description__proximity thunderstorm with rain,weather_description__scattered clouds,weather_description__sky is clear,weather_description__smoke,weather_description__snow,weather_description__thunderstorm,weather_description__thunderstorm with heavy rain,weather_description__thunderstorm with light rain,weather_description__thunderstorm with rain,weather_description__very heavy rain
0,2012-10-02 09:00:00,0,121,89,2,329,1,1,288.28,0.0,0.0,40,5545,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,2012-10-02 10:00:00,0,178,67,3,330,1,1,289.36,0.0,0.0,75,4516,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2012-10-02 11:00:00,0,113,66,3,329,2,2,289.58,0.0,0.0,90,4767,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2012-10-02 12:00:00,0,20,66,3,329,5,5,290.13,0.0,0.0,90,5026,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2012-10-02 13:00:00,0,281,65,3,329,7,7,291.14,0.0,0.0,75,4918,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
columns=['weather_type','weather_description']
dummies = [col for col in df
               if "__" in col 
               and col.split("__")[0] in columns]

In [0]:
X = df.drop('traffic_volume',axis=1)
y = df['traffic_volume']

In [0]:
normalizer = preprocessing.MinMaxScaler(feature_range = (0,1))
X_norm = normalizer.fit_transform(X.iloc[:,1:])

In [0]:
X_norm.shape

(33750, 50)

In [0]:
# for i in range(len(X_norm)):
#   X_norm[i,:] = X.iloc[i,0] + X_norm[i,:]

In [0]:
pre_X = []
pre_y = []
for i in range(10,len(y)):
  pre_X.append(X_norm[i-10:i,:])
  pre_y.append(y[i])

pre_X = np.array(pre_X)
pre_y = np.array(pre_y)

In [0]:
model = Sequential()
model.add(LSTM(units = 150,return_sequences=True,input_shape = (pre_X.shape[1],50)))
model.add(Dropout(0.3))
model.add(LSTM(units = 90,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units = 90,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
model.add(Dense(units=1,activation='linear'))


In [0]:
model.compile(optimizer='adam',loss = 'mean_squared_error',metrics = ['mean_absolute_error'])

In [0]:
from keras.callbacks import ModelCheckpoint

# checkpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_mean_absolute_error', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
model.fit(pre_X,pre_y,validation_split=0.2,batch_size=100,epochs = 100,callbacks=callbacks_list,shuffle=True,verbose=2)

W0805 13:46:42.327604 139879499077504 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 26992 samples, validate on 6748 samples
Epoch 1/100
 - 31s - loss: 14471468.2786 - mean_absolute_error: 3235.7156 - val_loss: 13888419.9440 - val_mean_absolute_error: 3173.1787

Epoch 00001: val_mean_absolute_error improved from inf to 3173.17870, saving model to weights.best.hdf5
Epoch 2/100
 - 27s - loss: 14378833.2948 - mean_absolute_error: 3221.4102 - val_loss: 13801928.8379 - val_mean_absolute_error: 3159.5211

Epoch 00002: val_mean_absolute_error improved from 3173.17870 to 3159.52106, saving model to weights.best.hdf5
Epoch 3/100
 - 27s - loss: 14291475.7999 - mean_absolute_error: 3207.8945 - val_loss: 13716650.4422 - val_mean_absolute_error: 3145.9966

Epoch 00003: val_mean_absolute_error improved from 3159.52106 to 3145.99662, saving model to weights.best.hdf5
Epoch 4/100
 - 27s - loss: 14201802.3740 - mean_absolute_error: 3193.9489 - val_loss: 13626709.5397 - val_mean_absolute_error: 3131.6695

Epoch 00004: val_mean_absolute_error improved from 3145.99662 to 3131.669

In [0]:
model.save('Used_Cars_model_1.h5')